<a href="https://colab.research.google.com/github/Servat0r/HLT-Project-2023/blob/master/AveragedCrossEntropy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd "/content/drive/MyDrive/Colab Notebooks"

/content/drive/MyDrive/Colab Notebooks


In [ ]:
%run utils.ipynb

Updated!


In [ ]:
from transformers import AutoModel, AutoTokenizer, MT5Model, MT5ForConditionalGeneration, AdamW, DataCollatorWithPadding
from datasets import load_dataset, Dataset, load_from_disk, load_metric
import numpy as np
import evaluate
import torch
import os

In [ ]:
class AveragedCrossEntropyLoss(torch.nn.CrossEntropyLoss):

  def __init__(
      self, weight = None, size_average=None, ignore_index: int = -100, sep_index: int = 1,  # <eos> token (tokenizer concatenates with <eos>)
      reduce=None, final_reduction: str = 'mean', label_smoothing: float = 0.0, num_references=4,
  ) -> None:
    super().__init__(
        weight, size_average=size_average, reduce=reduce,
        reduction='none', label_smoothing=label_smoothing
    )
    self.final_reduction = final_reduction
    self.sep_index = sep_index
    self.num_references = num_references

  def forward(self, predictions, targets):
    """
    TODO: WE ARE STILL NOT DOING A WEIGHTED AVERAGE!
    Predictions is of shape (batch_size, vocab_size).
    References is of shape (batch_size, num_references, vocab_size)
    Output is of shape (batch_size,)
    Intermediate result store is of shape (num_references, batch_size, seq_len)
    """
    batch_size, target_sequences_length = targets.shape[0], targets.shape[1] // self.num_references
    results = []
    for i in range(batch_size):
      current_prediction = predictions[i, :, :]
      intermediate = []
      for j in range(self.num_references):
        current_target = targets[i, j*target_sequences_length:(j+1)*target_sequences_length]
        intermediate.append(
            super().forward(current_prediction, current_target)
        )
      results.append(torch.mean(torch.stack(intermediate)))
    output = torch.stack(results)
    """
    intermediates = []
    for i in range(batch_size):
      if i % self.num_references == 0:
        intermediates.append([])
      prediction = predictions[i // self.num_references, :, :]
      target = targets[i, :]
      intermediates[-1].append(torch.mean(super().forward(prediction, target)))
    output = torch.tensor([torch.mean(torch.stack(results)) for results in intermediates], requires_grad=True)
    """
    if self.final_reduction == 'mean':
      return torch.mean(output)
    elif self.final_reduction == 'sum':
      return torch.sum(output)
    else:
      return output

In [ ]:
class MT5ForConditionalGenerationAverageCE(MT5ForConditionalGeneration):

  def __init__(self, *args, **kwargs):
    super().__init__(*args, **kwargs)
    self.loss = AveragedCrossEntropyLoss(final_reduction='mean', ignore_index=-100)  # TODO This implies having all references with same length!

  def forward(self, *, input_ids, attention_mask, labels=None, num_references=4, **kwargs):
    labels_length = labels.shape[1] // num_references
    ref_labels = labels[:, 0:labels_length].clone()
    base_output = super().forward(input_ids, attention_mask, labels=ref_labels, **kwargs)
    output = base_output
    output.loss = self.loss(base_output.logits, labels)
    return output